# Lab 2: Energy Balance and Surface Temperature

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MichalBrezny/LSPD-lab/blob/main/labs/Lab02_Energy_Balance_Temperature.ipynb)

## Objectives
- Understand radiation components and net radiation
- Calculate surface temperature from energy balance
- Analyze the relationship between albedo and surface temperature
- Explore the effects of cloud cover on radiation

## Background
The radiation budget is the foundation of the surface energy balance. Understanding how radiation interacts with the surface is crucial for predicting surface temperature and other land surface processes.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import optimize

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Physical constants
STEFAN_BOLTZMANN = 5.67e-8  # W m⁻² K⁻⁴
SOLAR_CONSTANT = 1367  # W m⁻²

print("Libraries imported successfully!")

## Part 1: Radiation Components

Net radiation is calculated as:

$R_n = (1 - \alpha) S_\downarrow + L_\downarrow - L_\uparrow$

where:
- $\alpha$ = surface albedo
- $S_\downarrow$ = incoming shortwave radiation
- $L_\downarrow$ = incoming longwave radiation
- $L_\uparrow$ = outgoing longwave radiation = $\epsilon \sigma T_s^4$

In [ ]:
def calculate_solar_radiation(hour, latitude=45, day_of_year=172):
    """
    Calculate incoming solar radiation based on time and location.
    Simplified model for educational purposes.
    
    Parameters:
    - hour: hour of day (0-24)
    - latitude: latitude in degrees
    - day_of_year: day of year (1-365)
    
    Returns:
    - Solar radiation in W/m²
    """
    # Solar declination
    declination = 23.45 * np.sin(2 * np.pi * (284 + day_of_year) / 365)
    
    # Hour angle
    hour_angle = 15 * (hour - 12)
    
    # Solar elevation angle (simplified)
    lat_rad = np.radians(latitude)
    dec_rad = np.radians(declination)
    ha_rad = np.radians(hour_angle)
    
    sin_elevation = (np.sin(lat_rad) * np.sin(dec_rad) + 
                     np.cos(lat_rad) * np.cos(dec_rad) * np.cos(ha_rad))
    
    # Solar radiation (W/m²)
    if isinstance(sin_elevation, np.ndarray):
        solar_rad = SOLAR_CONSTANT * np.maximum(0, sin_elevation)
    else:
        solar_rad = SOLAR_CONSTANT * max(0, sin_elevation)
    
    return solar_rad

def calculate_longwave_down(T_air, cloud_fraction=0):
    """
    Calculate incoming longwave radiation.
    
    Parameters:
    - T_air: air temperature in K
    - cloud_fraction: fraction of sky covered by clouds (0-1)
    
    Returns:
    - Longwave radiation in W/m²
    """
    # Clear-sky emissivity (simplified)
    emissivity_clear = 0.75
    emissivity_cloud = 0.95
    
    emissivity = emissivity_clear * (1 - cloud_fraction) + emissivity_cloud * cloud_fraction
    
    return emissivity * STEFAN_BOLTZMANN * T_air**4

def calculate_longwave_up(T_surface, emissivity=0.95):
    """
    Calculate outgoing longwave radiation.
    
    Parameters:
    - T_surface: surface temperature in K
    - emissivity: surface emissivity
    
    Returns:
    - Longwave radiation in W/m²
    """
    return emissivity * STEFAN_BOLTZMANN * T_surface**4

print("Radiation functions defined!")

In [ ]:
# Create hourly data for a summer day
hours = np.arange(0, 24, 0.5)

# Calculate radiation components
SW_down = np.array([calculate_solar_radiation(h, latitude=45, day_of_year=172) for h in hours])

# Air temperature variation
T_air_C = 20 + 8 * np.sin(np.pi * (hours - 8) / 12)
T_air_K = T_air_C + 273.15

# Longwave down (no clouds)
LW_down = calculate_longwave_down(T_air_K, cloud_fraction=0)

# Different surface types
surfaces = {
    'Grass': {'albedo': 0.20, 'emissivity': 0.95},
    'Forest': {'albedo': 0.15, 'emissivity': 0.97},
    'Desert': {'albedo': 0.30, 'emissivity': 0.90},
    'Snow': {'albedo': 0.70, 'emissivity': 0.98}
}

# Assume surface temperature ~ air temperature + some offset (simplified)
T_surf_offset = 5 * np.sin(np.pi * (hours - 6) / 12)
T_surf_offset[T_surf_offset < 0] = 0

# Calculate for each surface
results = {}
for surface_name, props in surfaces.items():
    T_surf_K = T_air_K + T_surf_offset
    SW_net = (1 - props['albedo']) * SW_down
    LW_up = calculate_longwave_up(T_surf_K, props['emissivity'])
    LW_net = LW_down - LW_up
    Rn = SW_net + LW_net
    
    results[surface_name] = {
        'SW_net': SW_net,
        'LW_net': LW_net,
        'Rn': Rn,
        'T_surf': T_surf_K - 273.15
    }

print("Radiation components calculated for all surfaces!")

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Incoming radiation components
ax = axes[0, 0]
ax.plot(hours, SW_down, 'orange', linewidth=2, label='Shortwave Down')
ax.plot(hours, LW_down, 'red', linewidth=2, label='Longwave Down')
ax.set_xlabel('Hour of Day', fontsize=11)
ax.set_ylabel('Radiation (W/m²)', fontsize=11)
ax.set_title('Incoming Radiation Components', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 2: Net radiation for different surfaces
ax = axes[0, 1]
for surface_name, data in results.items():
    ax.plot(hours, data['Rn'], linewidth=2, label=surface_name)
ax.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
ax.set_xlabel('Hour of Day', fontsize=11)
ax.set_ylabel('Net Radiation (W/m²)', fontsize=11)
ax.set_title('Net Radiation by Surface Type', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 3: Shortwave net by surface
ax = axes[1, 0]
for surface_name, data in results.items():
    ax.plot(hours, data['SW_net'], linewidth=2, label=surface_name)
ax.set_xlabel('Hour of Day', fontsize=11)
ax.set_ylabel('Net Shortwave (W/m²)', fontsize=11)
ax.set_title('Net Shortwave by Surface Type (Albedo Effect)', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 4: Surface temperature
ax = axes[1, 1]
ax.plot(hours, T_air_C, 'k--', linewidth=2, label='Air Temperature')
for surface_name, data in results.items():
    ax.plot(hours, data['T_surf'], linewidth=2, label=surface_name)
ax.set_xlabel('Hour of Day', fontsize=11)
ax.set_ylabel('Temperature (°C)', fontsize=11)
ax.set_title('Surface Temperature by Surface Type', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Part 2: Effect of Cloud Cover

Clouds significantly affect both incoming shortwave (by reflection) and longwave (by emission) radiation.

In [ ]:
# Different cloud conditions
cloud_fractions = [0, 0.3, 0.6, 1.0]
cloud_results = {}

for cf in cloud_fractions:
    # Reduce solar radiation based on cloud cover
    SW_down_cloudy = SW_down * (1 - 0.6 * cf)
    
    # Increase longwave down based on cloud cover
    LW_down_cloudy = calculate_longwave_down(T_air_K, cloud_fraction=cf)
    
    # Calculate net radiation for grass surface
    albedo = 0.20
    SW_net = (1 - albedo) * SW_down_cloudy
    
    # Simplified surface temperature (would need iterative solution in reality)
    T_surf_K = T_air_K + T_surf_offset
    LW_up = calculate_longwave_up(T_surf_K, emissivity=0.95)
    LW_net = LW_down_cloudy - LW_up
    Rn = SW_net + LW_net
    
    cloud_results[f'{int(cf*100)}%'] = {
        'SW_down': SW_down_cloudy,
        'LW_down': LW_down_cloudy,
        'Rn': Rn
    }

print("Cloud effect calculations complete!")

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Plot shortwave
ax = axes[0]
for label, data in cloud_results.items():
    ax.plot(hours, data['SW_down'], linewidth=2, label=f'Clouds: {label}')
ax.set_xlabel('Hour of Day', fontsize=11)
ax.set_ylabel('Shortwave Down (W/m²)', fontsize=11)
ax.set_title('Effect of Clouds on Incoming Shortwave', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot longwave
ax = axes[1]
for label, data in cloud_results.items():
    ax.plot(hours, data['LW_down'], linewidth=2, label=f'Clouds: {label}')
ax.set_xlabel('Hour of Day', fontsize=11)
ax.set_ylabel('Longwave Down (W/m²)', fontsize=11)
ax.set_title('Effect of Clouds on Incoming Longwave', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot net radiation
ax = axes[2]
for label, data in cloud_results.items():
    ax.plot(hours, data['Rn'], linewidth=2, label=f'Clouds: {label}')
ax.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
ax.set_xlabel('Hour of Day', fontsize=11)
ax.set_ylabel('Net Radiation (W/m²)', fontsize=11)
ax.set_title('Effect of Clouds on Net Radiation', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Exercises

### Exercise 1: Albedo Sensitivity
Calculate how much the daily total net radiation changes when albedo increases from 0.2 to 0.3.

In [ ]:
# Your code here
albedo1 = 0.2
albedo2 = 0.3

Rn1 = np.sum((1 - albedo1) * SW_down) * 0.5  # 0.5 hour timestep
Rn2 = np.sum((1 - albedo2) * SW_down) * 0.5

difference = Rn1 - Rn2
percent_change = (difference / Rn1) * 100

print(f"Daily net shortwave with albedo {albedo1}: {Rn1:.0f} Wh/m²")
print(f"Daily net shortwave with albedo {albedo2}: {Rn2:.0f} Wh/m²")
print(f"Difference: {difference:.0f} Wh/m² ({percent_change:.1f}% reduction)")

### Exercise 2: Radiation Balance at Night
Calculate the nighttime (when SW_down = 0) radiation balance and explain the result.

In [ ]:
# Your code here
night_hours = hours[(hours < 6) | (hours > 18)]
night_idx = (hours < 6) | (hours > 18)

# For grass surface
night_Rn = results['Grass']['Rn'][night_idx]

print(f"Average nighttime net radiation: {np.mean(night_Rn):.1f} W/m²")
print(f"\nInterpretation: Negative net radiation at night indicates")
print(f"the surface is losing energy through longwave emission,")
print(f"which causes nighttime cooling.")

## Discussion Questions

1. **Why does snow have a much different net radiation pattern than other surfaces?**
   - High albedo reflects most solar radiation
   - Results in lower net shortwave and cooler temperatures

2. **How do clouds affect the diurnal temperature range?**
   - Reduce daytime maximum by blocking solar radiation
   - Increase nighttime minimum by emitting longwave radiation
   - Overall effect: reduced diurnal temperature range

3. **What would happen to surface temperature if we removed all vegetation (changed albedo)?**
   - Depends on new surface albedo
   - Bare soil typically has higher albedo than vegetation
   - Also affects moisture availability and evapotranspiration

4. **Why is longwave radiation relatively constant throughout the day?**
   - Depends on atmospheric temperature and emissivity
   - Atmospheric temperature has smaller diurnal variation
   - Cloud cover can cause variations

## Summary

In this lab, you have:
- Calculated radiation components (shortwave and longwave)
- Analyzed the effect of surface albedo on net radiation
- Explored how cloud cover affects the radiation balance
- Connected radiation balance to surface temperature

## Next Steps
In Lab 3, we will examine soil moisture dynamics and their interaction with the energy balance.